In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train_raw = pd.read_csv("cnndm/train_raw.csv", sep = ";")
test_raw = pd.read_csv("cnndm/test_raw.csv", sep = ";")
val_raw = pd.read_csv("cnndm/valid_raw.csv", sep = ";")

In [3]:
# Load the tokenizer and model
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")

In [ ]:
from functions.fine_tune_model import filter_rows_by_length

filtered_train = filter_rows_by_length(train_raw, tokenizer, max_length=512)
filtered_test = filter_rows_by_length(test_raw, tokenizer, max_length=512)
filtered_val = filter_rows_by_length(val_raw, tokenizer, max_length=512)

In [ ]:
# Save filtered datasets
filtered_train.to_csv("cnndm/subsamples/train_filtered.csv", sep=";", index=False)
filtered_test.to_csv("cnndm/subsamples/test_filtered.csv", sep=";", index=False)
filtered_val.to_csv("cnndm/subsamples/valid_filtered.csv", sep=";", index=False)

print("Datasets filtered and saved.")